<a href="https://colab.research.google.com/github/ClaireChiang91/NLP_MSUcourse/blob/main/NLP_HW1_Part4_N_Gram_language_Models_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program is going to do with the assignment for the NLP course at MSU. 
https://github.com/deskool/nlp-class
Homework1 Learning Excerse 4:
https://github.com/deskool/nlp-class/blob/master/homework/HW1/assignment.ipynb

**N-grams**
An n-gram is a contiguous sequence of n items from a given sample of text or speech. Let's begin this section by defining a function that generates n-grams from our text data using the nltk library and using it to extract n-grams for n ranging from 1-5.
**The challenge**
Now that you have built your first language model, let's turn our attention back to the philosophers. For this final learning exercise, you will be building a tri-gram Nietzsche language model, and a tri-gram Russel Language Model using the 5 books from each of the philosophers you collected from Project Gutenburg earlier.

Generate a conversation between your philosophers models by seeding one of the models with a tri-gram and allow it to run until it generates a terminating character (a '.', '?' or '!'). Then, have the other model generate a response by providing the ending tri-gram of the the first model as the seed tri-gram for the other model. Iterate like this for 15 sentences and print the text.

Note: You probably learned from the earlier components of this assignment that Nietzsche and Russel have slightly different writing styles (yes, that was an understatement). For this reason, there are likely to be instances where the ending tri-gram from your Nietzsche model does not show up in the Russel language model and vice-versa. Use Laplace smoothing to handle this problem.

In [1]:
import requests
import numpy as np
import re

import nltk
from nltk.util import ngrams
#to solve the error msg: Resource punkt not found. 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Nietzsche
website = 'http://www.gutenberg.org/cache/epub/4363/pg4363.txt'
nietz    = requests.get(website).text[:200]

# Russel
website = 'http://www.gutenberg.org/cache/epub/5827/pg5827.txt'
russel = requests.get(website).text[:200]
########################################################################
#to apply the request of iterating 15 sentences
max_setence=15 
#k smoothing, which could be 0.05, 0.5 or 1; 1 is to apply the reuqest of using Laplace
k=1
#stands for n-grams
n=3

In [3]:
#define an ngram extractor
def extract_word_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [4]:
def language_model(philosopher,size,k):
  grams=extract_word_ngrams(philosopher, size)
  vocabulary = list(set(grams))
  counts = {}
  for given_word in vocabulary:
    counts[given_word] = {}
    for next_word in vocabulary:
        counts[given_word][next_word] = 0 

  for i in range(len(grams)-1):
    counts[grams[i]][grams[i+1]] += 1

  # initialize the probabilites JSON object
  probs = {}
  for given_word in vocabulary:
    probs[given_word] = {}
    for next_word in vocabulary:
      probs[given_word][next_word] = 0

  # convert the counts to probabilites
  for key, value in counts.items():
    denominator = 0
    for key2, value2 in counts[key].items():
      denominator += value2
    for key2, value2 in counts[key].items():
      #Add-k smoothing
      probs[key][key2] = (value2+k) / (denominator+k*len(vocabulary))
  return probs

In [5]:
def sample_next_gram_from_language_model(probs, given_token):
    distribution            = list(probs[given_token].values())
    sample_from_multinomial = np.random.multinomial(1,distribution)
    sample_index            = np.where(sample_from_multinomial==1)[0][0]
    word_keys               = list(probs[given_token].keys())
    next_word               = word_keys[sample_index]
    return(next_word)

In [6]:
def create_new_sentence(_probs_,seed_token):
    tokens = [seed_token]
    while any([' '.join(tokens).split(' ')[-1] not in ['.', '?','!'] ]):
      _next_word_=sample_next_gram_from_language_model(_probs_,tokens[-1])
      if all([_next_word_.split(' ')[i] not in ['.', '?','!'] for i in range(len(_next_word_.split(' ')))]):
        tokens.append(_next_word_)
      else:
        i_index=0
        while i_index < len(_next_word_.split(' ')):
          _nextword_=_next_word_.split(' ')[i_index]
          i_index+=1
          if _nextword_ not in ['.', '?','!']:
            tokens.append(_nextword_)
          else:  
            tokens.append(_nextword_)
            break
    return tokens

In [7]:
nietz_probs=language_model(nietz, n,k)
seed_token_nietz= 'Good and Evil'
' '.join(create_new_sentence(nietz_probs, seed_token_nietz))

'Good and Evil cost and with This eBook is it , g no restrictions whatsoever Project Gutenberg EBook use of anyone is for the and with almost Beyond Good and Project Gutenberg EBook no restrictions whatsoever Beyond Good and Good and Evil EBook of Beyond at no cost of Beyond Good and with almost by Friedrich Nietzsche copy it , Friedrich Nietzsche This almost no restrictions anywhere at no Nietzsche This eBook the use of for the use at no cost may copy it may copy it .'

In [8]:
russel_probs=language_model(russel, n,k)
seed_token_russel= 'no cost and'

' '.join(create_new_sentence(russel_probs, seed_token_russel))

"no cost and This eBook is Problems of Philosophy 's The Problems with almost no by Bertrand Russell Bertrand Russell This , by Bertrand Gutenberg 's The ."

In [22]:
def conversation(seed_p,max_setence,_initialseed_):
  initial_set=0
  #opening
  if initial_set==0:
    if seed_p=='nietz':
      #initial_seed='Good and Evil'
      response=' '.join(create_new_sentence(nietz_probs, _initialseed_))
      print("counts:",initial_set)
      print(seed_p,"initially say:",response)
      print("=========")  
      initial_set+=1
      seed_p='russel'
    else:
      #initial_seed='no cost and'
      response=' '.join(create_new_sentence(russel_probs, _initialseed_))
      print("counts:",initial_set)
      print(seed_p,"initially say:",response)
      print("=========")
      initial_set+=1
      seed_p='nietz'

  #response
  while initial_set<=max_setence:
    next_seed=' '.join(response.split(" ")[-3:])
    if seed_p=='nietz':    
      if next_seed in nietz_probs.keys():
        response=' '.join(create_new_sentence(nietz_probs, next_seed))
        print("counts:",initial_set)
        print(seed_p,"responds:",response)
        print("=========")
        initial_set+=1
        seed_p='russel'
      else:
        #starting for loop to look for the last three words in the dic
        for i in range(0,len(response.split(" "))-3):
          next_seed=' '.join(response.split(" ")[(-4-i):(-1-i)])
          if next_seed in nietz_probs.keys():
            response=' '.join(create_new_sentence(nietz_probs, next_seed))
            print("counts:",initial_set)
            print(seed_p,"responds:",response)
            print("=========")
            initial_set+=1
            seed_p='russel'
            break
          elif i==len(response.split(" "))-3-1 and next_seed not in nietz_probs.keys():
            print("counts:",initial_set)
            print(seed_p,"responds: no respone")
            print("===end======")
            initial_set+=1
            seed_p='russel'

    else: #the scenario of seed_p=='russel'
      if next_seed in russel_probs.keys():
        response=' '.join(create_new_sentence(russel_probs, next_seed))
        print("counts:",initial_set)
        print(seed_p,"responds:",response)
        print("=========")
        initial_set+=1
        seed_p='nietz'
      else:
        for i in range(0,len(response.split(" "))-3):
          next_seed=' '.join(response.split(" ")[(-4-i):(-1-i)])
          if next_seed in russel_probs.keys():
            response=' '.join(create_new_sentence(russel_probs, next_seed))
            print("counts:",initial_set)
            print(seed_p,"responds:",response)
            print("=========")
            initial_set+=1
            seed_p='nietz'
            break
          elif i==len(response.split(" "))-3-1 and next_seed not in russel_probs.keys():
            print("counts:",initial_set)
            print(seed_p,"responds: no respone")
            print("===end======") 
            initial_set+=1
            seed_p='nietz'

In [23]:
conversation('nietz',max_setence,'Good and Evil' )

counts: 0
nietz initially say: Good and Evil Nietzsche This eBook Project Gutenberg EBook Good and Evil restrictions whatsoever .
counts: 1
russel responds: restrictions whatsoever .
counts: 2
nietz responds: restrictions whatsoever .
counts: 3
russel responds: restrictions whatsoever .
counts: 4
nietz responds: restrictions whatsoever .
counts: 5
russel responds: restrictions whatsoever .
counts: 6
nietz responds: restrictions whatsoever .
counts: 7
russel responds: restrictions whatsoever .
counts: 8
nietz responds: restrictions whatsoever .
counts: 9
russel responds: restrictions whatsoever .
counts: 10
nietz responds: restrictions whatsoever .
counts: 11
russel responds: restrictions whatsoever .
counts: 12
nietz responds: restrictions whatsoever .
counts: 13
russel responds: restrictions whatsoever .
counts: 14
nietz responds: restrictions whatsoever .
counts: 15
russel responds: restrictions whatsoever .
